<a href="https://colab.research.google.com/github/karaage0703/karaage-ai-book/blob/master/ch02/02_karaage_ai_book_image_classification_tf2_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification by deep-learning (AI)

This is an exercise to practice Image Classification of Janken (Rock-Paper-Scissors) hand shape.

## Download Training Data

Download (Clone) the "Janken" Training data from GitHub.

In [ ]:
!git clone https://github.com/fu11ji26/janken_dataset datasets
!rm -rf datasets/.git

Check the contents of data and display the image of hand shape, "gu"(=Rock), "choki" (=Scissors) or "pa"(=Paper).

In [ ]:
# !ls datasets/gu
!ls datasets/choki
# !ls datasets/pa

In [ ]:
from IPython.display import Image as IPImage
from IPython.display import display_jpeg
display_jpeg(IPImage('datasets/choki/IMG_0770.JPG'))

## Split the original training data into Training Data and Validation Data.

Visualize the folder structure with a software called **tree**.

In [ ]:
!sudo apt install tree
!tree -d datasets

Make an original data sets folder and a target folder.

The target folder consists of the training data sets and validation data sets.

In [ ]:
dataset_original_dir = 'datasets'
dataset_root_dir = 'target_datasets'

Split the target datasets into training and validation data.

The ratio of training data to validation data is specified by "train_size" which default value is 0.8.

In [ ]:
!wget "https://raw.githubusercontent.com/fu11ji26/janken-classification-lesson/master/split_train_val.py" -O "split_train_val.py"
import split_train_val
split_train_val.image_dir_train_val_split(dataset_original_dir, dataset_root_dir, train_size=0.8)
train_dir = 'target_datasets/train'
val_dir = 'target_datasets/val'

Visualize the folder structure and check the contens of data.

In [ ]:
!tree -d target_datasets

In [ ]:
!ls target_datasets/train/choki

In [ ]:
!ls target_datasets/val/choki

## Create a label files
Label names are choki, gu and pa.

学習するファイルのラベルを作成します

First, import the required libraries.

Then, specify the back-up folder to save the data.

Third, create the label data.

Last, display the class number which is the number of images (= gu, choki, pa).


In [ ]:
import sys
import os
import shutil
backup_dir = './model'

In [ ]:
labels = [d for d in os.listdir(dataset_original_dir) \
    if os.path.isdir(os.path.join(dataset_original_dir, d))]
labels.sort()

if os.path.exists(backup_dir):
  shutil.rmtree(backup_dir)

os.makedirs(backup_dir)

with open(backup_dir + '/labels.txt','w') as f:
  for label in labels:
    f.write(label+"\n")

NUM_CLASSES = len(labels)
print("class number=" + str(NUM_CLASSES))

Check the labels. If the label names (choki, gu, pa) are lined up, it is OK.

In [ ]:
!cat ./model/labels.txt

## Set up the learning

Import the necessary libraries.

Google colaboratory works with "TensorFlow 2.x" series, so select the version 2.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

Load label information from the label files created ahead.

In [ ]:
labels = []
with open(backup_dir + '/labels.txt','r') as f:
  for line in f:
    labels.append(line.rstrip())
print(labels)

NUM_CLASSES = len(labels)

### Set hyper-parameters for learning

In [ ]:
# Learning rate
LEARNING_RATE = 0.001
# Epoch (number of generations)
EPOCHS = 20
# batch size
BATCH_SIZE = 8

### Pre-process (transform) the data set

In [ ]:
IMAGE_SIZE = 64

train_data_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = ImageDataGenerator(rescale=1./255)

train_data = train_data_gen.flow_from_directory(
    train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb', batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True)

validation_data = val_data_gen.flow_from_directory(
    val_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb', batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True)

### Check training data and labels of learning for one unit (batch) size

In [ ]:
(image_data,label_data) = train_data.next()
print(image_data)
print(label_data)

Size of training data and labels for one unit

In [ ]:
print(image_data.shape)
print(label_data.shape)

In [ ]:
import matplotlib.pyplot as plt
image_numb = 3 # Specify 3 or 6
for i in range(0, image_numb):
  ax = plt.subplot(image_numb // 3, 3, i + 1)
  plt.tight_layout()
  ax.set_title(str(i))
  plt.imshow(image_data[i])

## Creating AI Models
Create a convolutional neural network (CNN) model.

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
#opt = tf.keras.optimizers.SGD(lr=LEARNING_RATE)

model.compile(opt, loss='categorical_crossentropy', 
    metrics=['accuracy'])

Review the model overview.

The total number of parameters in the network is about 8 million.

In [ ]:
model.summary()

Performs AI model training.

In [ ]:
%%time
history = model.fit(train_data, epochs=EPOCHS, validation_data=validation_data, verbose=1)

## Visualization of learning results
Check the "loss" (value of the error function).

The lower the loss, the better the performance.

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlim([0.0, EPOCHS])
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()

Check the accuracy.

"acc" is accuracy for training data.

For example, "acc = 0.8" means 80% correct.

The so-called true accuracy is "val_acc" which is accuracy for validation (non-trained) data.

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and validation accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.xlim([0.0, EPOCHS])
plt.ylim([0.0, 1.0])
plt.legend(['acc', 'val_acc'], loc='lower right')
plt.show()

## Estimate the hand shape of validation (non-trained) data

In [ ]:
# Get the ordered list of class names:
import PIL.Image as Image
class_names = validation_data.class_indices.items()
class_names = np.array([key.title() for key, value in class_names])

validation_data.reset()
validation_data.shuffle = True
validation_data.batch_size = BATCH_SIZE

# Retrieve the first batch from the validation data
for validation_image_batch, validation_label_batch in validation_data:
  break

validation_id = np.argmax(validation_label_batch, axis=-1)
validation_label = class_names[validation_id]
predicted_batch = model.predict(validation_image_batch)

# Returns the indices of the maximum values along a given axis
predicted_id = np.argmax(predicted_batch, axis=-1)

# Return the maximum values along a given axis
predicted_score = np.max(predicted_batch, axis=-1)

predicted_label_batch = class_names[predicted_id]

plt.figure(figsize=(16, 9))
plt.subplots_adjust(hspace=0.5)

# Display the classification results for the first 30 images
for n in range(min(validation_image_batch.shape[0], 30)):
  plt.subplot(6, 5, n + 1)

  # Convert the range from -1 to 1 to the range from 0 to 1
  plt.imshow(np.array(validation_image_batch[n]*255,np.int32))
  color = 'green' if predicted_id[n] == validation_id[n] else 'red'
  predicted_label = predicted_label_batch[n].title()
  plt.title(predicted_label + ' ({:.2f}, {})'.format(
      predicted_score[n], validation_label[n]), color=color)
  plt.axis('off')

_ = plt.suptitle('Model predictions (green: correct, red: incorrect)')

Displays the results of evaluating the performance of the AI model by Confusion Matrix.

Confusion Matrix represents the ratio of the model's predicted result to the correct answers.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

validation_data.reset()
validation_data.shuffle =  False
validation_data.batch_size = 1

# Retrieve the first batch from the validation data
for validation_image_batch, validation_label_batch in validation_data:
  break

predicted = model.predict(validation_data, steps=validation_data.n)
predicted_classes = np.argmax(predicted, axis=-1)

# Apply normalization
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
cm = confusion_matrix(validation_data.classes, predicted_classes)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 9))

# https://seaborn.pydata.org/generated/seaborn.heatmap.html
# https://matplotlib.org/users/colormaps.html
sns.heatmap(cm, annot=True, square=True, cmap=plt.cm.Blues,
            xticklabels=validation_data.class_indices,
            yticklabels=validation_data.class_indices)

plt.title("Confusion Matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.xlim([0.0, 3.0])
plt.ylim([0.0, 3.0])
plt.show()

## Summary


At this point, the lesson of steps from data preparation, training and  estimation is complete.

Let's change the hyper-parameters and see what happens to change in results of learning.

## References (almost in Japanese)

Structures of Convolutional Newral Network
- http://aidiary.hatenablog.com/entry/20161127/1480240182

Data Augmentation
- https://github.com/bohemian916/deeplearning_tool/blob/master/increase_picture.py

GradCam
- https://github.com/shinmura0/Python-study-group/blob/master/Text3.ipynb

GradCam Confusion Matrix
- https://colab.research.google.com/drive/1mirG8BSoB3k87mh-qyY3-8-ZXj0XB6h6

Support for TensorFlow 2.x
- http://tensorflow.classcat.com/2019/11/04/tf20-tutorials-images-classification/

Fixing random numbers (Seeds)
- https://scrapbox.io/nwtgck/Tensorflow+Keras%E3%81%A7%E5%86%8D%E7%8F%BE%E6%80%A7%E3%81%AE%E3%81%82%E3%82%8B%E4%B9%B1%E6%95%B0%E3%82%92%E7%94%9F%E6%88%90%E3%81%99%E3%82%8B_-_%E3%82%B7%E3%83%BC%E3%83%89%E5%9B%BA%E5%AE%9A
- https://qiita.com/okotaku/items/8d682a11d8f2370684c9


Other Related Information
- http://aidiary.hatenablog.com/entry/20161212/1481549365
- https://qiita.com/yampy/items/706d44417c433e68db0d
- https://qiita.com/haru1977/items/17833e508fe07c004119
- http://hatakazu.hatenablog.com/entry/2017/06/08/045953
- https://qiita.com/Mco7777/items/2b76aba1bae35f2623ea

